In [6]:
import psycopg2
import sys
import pprint
from datetime import date, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import os,sys
import matplotlib.patches as mpatches

#sys.path.append("../Mfig/")
#import mplp
from itertools import chain
import numpy as np
import csv
from collections import defaultdict
from datetime import date, timedelta
from datetime import datetime
from scipy.stats import entropy
%matplotlib inline

## DB connection

In [7]:
def conectionDB():
    conn_string = "dbname='stic' port='5432' user='stic' password='stic2019' host='gpmaster.as-dell.copernic.local'"
    #print ("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string)
    return conn

# build dictionaries


In [8]:
dict_dpto = dict()
dict_province = dict()
dict_district = dict()
with open('../data/ubigeo_inei.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        #print (row)
        if (str(row[0])!=''):
            dict_dpto[str(row[3]).lower()]=(str(row[0]))
        if (str(row[1])!=''):
            dict_province[(row[4]).lower()]=(row[0]+''+row[1])
        if (str(row[2])!=''):
            dict_district[(row[5]).lower()]=(row[0]+''+row[1]+''+row[2])
        
dict_gender = {'M':1,'F':0}
dict_card = {'TD':1,'TC':0}
dict_region = {'NORTE':0,'ORIENTE':1,'SUR':2,'CENTRO':3,'SIERRA CENTRAL':4,
               'LIMA CENTRO':5,'LIMA ESTE':6,'LIMA SUR':7,'LIMA NORTE':8,
               'LIMA MODERNA':9,'LIMA PROVINCIA':10,'CALLAO':11,'':12}


In [9]:
columnNames=['client_id',    #-- 0  
        'social_class' ,     #-- 1
        'date',              #-- 2
        'mcc',               #-- 3
        'country_code',      #-- 4
        'amount_usd',        #-- 5 *
        'client_age',        #-- 6 *
        'client_gender',     #-- 7
        'debit_type',        #-- 8 Credit or debit card
        'agency_id',         #-- 9
        'agency_departement',#-- 10 
        'agency_province',   #-- 11
        'agency_district',   #-- 12
        'agency_lima',       #-- 13
        'agency_region',     #-- 14
        'merchant_id',       #-- 15
        'merchant_departement', #-- 16
        'merchant_province', #-- 17
        'merchant_district', #-- 18
        'merchant_lon',      #-- 19 *
        'merchant_lat']


## extraction loop

In [10]:
def extract_all (index):
    PLIMIT = 1000000
    POFFSET = 1000000
    #NUM = 93 # Millions of records (92 470 071)
    NUM = 5 #for test 
    i=index
    while (i < NUM):
        print ("Processing {}/{} ...".format(i,NUM))
        query ="""
            SELECT 
                bbva.client_id,    -- 0  
                social_class ,     -- 1
                date,              -- 2
                mcc,               -- 3
                country_code,      -- 4
                amount_usd,        -- 5 *
                client_age,        -- 6 *
                client_gender,     -- 7
                debit_type,        -- 8 Credit or debit card
                agency_id,         -- 9
                agency_departement,-- 10 
                agency_province,   -- 11
                agency_district,   -- 12
                agency_lima,       -- 13
                agency_region,     -- 14
                merchant_id,       -- 15
                merchant_departement, -- 16
                merchant_province, -- 17
                merchant_district, -- 18
                merchant_lon,      -- 19 *
                merchant_lat      -- 20 *
            FROM 
                public.bbva 
            INNER JOIN 
                client
            ON 
                client.client_id = bbva.client_id
            ORDER BY 
                bbva.date ASC
            LIMIT {}
            OFFSET {}
        ;
        """.format(PLIMIT,POFFSET*i)
        conn = conectionDB()
        cursor = conn.cursor();
        df = pd.read_sql(query, conn)
        
        df['small_social_class'] = df.apply(lambda row: 1 if (row.social_class<=3) else (3 if (row.social_class>=7) else 2), axis=1)
        del df['social_class']
        df['country_code'] = df.apply(lambda row: 1 if (row.country_code=='PE') else 0 , axis=1)
        df['agency_lima'] = df.apply(lambda row: 1 if (row.agency_lima=='TRUE') else 0 , axis=1)
        df["date"] = pd.to_datetime(df["date"],infer_datetime_format=True)
        df["date"]=df.apply(lambda row: row.date.timestamp() , axis=1) 
        df["client_id"]=df.apply(lambda row: sum([ord(i) for i in str(row.client_id)]) , axis=1) 


        df=df.replace({"agency_region":dict_region,
                       "client_gender": dict_gender,
                          "debit_type":dict_card,
                         "agency_departement":dict_dpto,
                         "agency_province":dict_province,
                         "agency_district":dict_district,
                          "merchant_departement":dict_dpto,
                         "merchant_province":dict_province,
                         "merchant_district":dict_district
                         })
        
        df["agency_departement"] = df["agency_departement"].str.lower()
        df["agency_province"] = df["agency_province"].str.lower()
        df["agency_district"] = df["agency_district"].str.lower()
        df["merchant_departement"] = df["merchant_departement"].str.lower()
        df["merchant_province"] = df["merchant_province"].str.lower()
        df["merchant_district"] = df["merchant_district"].str.lower()

        df.fillna(-1, inplace=True)

        df.to_csv (r'../data_output/export_dataframe_{}.csv'.format(i), index = None, header=False)
        i+=1

In [11]:
extract_all(1)

Processing 1/5 ...
Processing 2/5 ...
Processing 3/5 ...
Processing 4/5 ...
